<a href="https://colab.research.google.com/github/orbenishay/pandas-project/blob/master/Bicycle_6_Hagai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt # visualization, EDA
import numpy as np  # Mathematical calculations
%matplotlib inline


In [0]:
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

In [24]:
# Hagai's Analysis:
# -----------------------------------------------------------------------------------------
# Read csv's
df_trip = pd.read_csv("trip.csv",
                   index_col='id', parse_dates=True)
df_trip['Date_ec'] = pd.to_datetime(df_trip['start_date'], errors='coerce')
df_trip['date_day']=df_trip['Date_ec'].dt.date
df_trip.head()

df_station = pd.read_csv("station.csv", index_col = 'id')
df_station.head()

df_weather = pd.read_csv("weather.csv", parse_dates=['date'])
df_weather['date_ec'] = pd.to_datetime(df_weather['date'], errors='coerce')
df_weather['date_day']=df_weather['Date_ec'].dt.date
df_weather.head()

# -----------------------------------------------------------------------------------------
# Merges
df_trip_weather = df_trip.merge(df_weather, how='left', left_on = 'date_day', right_on='date_day')
df_trip_weather.head()

# ----------------------------------------------------------------------------------------
# Year/Month aggregation
df_trip['day']=df_trip['start_date']
df_trip['year'] = pd.DatetimeIndex(df_trip['start_date']).year
df_trip['month'] = pd.DatetimeIndex(df_trip['start_date']).month
df_trip['day'] = pd.DatetimeIndex(df_trip['start_date']).day
df_trip['hour'] = pd.DatetimeIndex(df_trip['start_date']).hour
df_trip['weekday'] = pd.DatetimeIndex(df_trip['start_date']).weekday
df_group=df_trip.groupby(['month','year']).agg({'start_date':'count'}).unstack(-1)
df_group.plot(kind='bar', stacked=False, figsize=(12, 6), by='year')
df_group.head(20)
df_trip['start']=pd.to_datetime(df_trip['start_date'])
df_trip['end']=pd.to_datetime(df_trip['end_date'])
df_trip['duration']=df_trip['end']-df_trip['start']
df_trip.head()

# ------------------------------------------------------------------------------------------
# rents distribution by city
df_station = pd.read_csv("station.csv", index_col = 'id')
df_trip_station = df_trip.merge(df_station, how='left', left_on = 'start_station_id', right_on='id')
# df_trip_station.head()
df_group_city_station=df_trip_station.groupby(['city','station_name']).agg({'start_date':'count'})
df_group_city_station.boxplot(column='start_date', by='city')
df_group_city_station.groupby('city').describe().unstack(0)
df_group_city_station.head()

# ------------------------------------------------------------------------------------------
# count rent by hour & weekday

# ------------------------------------------------------------------------------------------
# duration distribution
df_trip['start']=pd.to_datetime(df_trip['start_date'])
df_trip['end']=pd.to_datetime(df_trip['end_date_orig'])
df_trip['duration']=df_trip['end']-df_trip['start']
df_station = pd.read_csv("station.csv", index_col = 'id')
df_trip_station = df_trip.merge(df_station, how='left', left_on = 'start_station_id', right_on='id')
df_trip_station.boxplot(column='duration', by='city')
df_trip_station['duration'].astype('timedelta64[s]').plot.hist()


df_trip_station['duration_minutes'] = pd.DatetimeIndex(df_trip_station['duration'])
df_trip_station.groupby(['city']).agg({'duration_minutes':'mean'})


.hist(column='duration', by='city', bins=50)
df['time_delta'].astype('timedelta64[s]').plot.hist()





,start_station_id,end_station_id,bike_id,subscription_type,zip_code_x,start_date,end_date_orig,end_date,Date_ec_x,date_day,date,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code_y,Date_ec_y,date_ec
0,75,66,626,Subscriber,94107,29/08/2013 11:43,29/08/2013 11:57,8/29/2013 12:12,2013-08-29 11:43:00,2013-08-29,2013-08-29,68.0,75.0,10.0,11.0,0,4.0,NaN,286.0,94107.0,2013-08-29,2013-08-29
1,75,66,626,Subscriber,94107,29/08/2013 11:43,29/08/2013 11:57,8/29/2013 12:12,2013-08-29 11:43:00,2013-08-29,2013-08-29,71.0,79.0,10.0,6.0,0,5.0,NaN,313.0,94063.0,2013-08-29,2013-08-29
2,75,66,626,Subscriber,94107,29/08/2013 11:43,29/08/2013 11:57,8/29/2013 12:12,2013-08-29 11:43:00,2013-08-29,2013-08-29,71.0,71.0,10.0,8.0,0,4.0,NaN,355.0,94301.0,2013-08-29,2013-08-29
3,75,66,626,Subscriber,94107,29/08/2013 11:43,29/08/2013 11:57,8/29/2013 12:12,2013-08-29 11:43:00,2013-08-29,2013-08-29,70.0,72.0,10.0,5.0,0,4.0,NaN,355.0,94041.0,2013-08-29,2013-08-29
4,75,66,626,Subscriber,94107,29/08/2013 11:43,29/08/2013 11:57,8/29/2013 12:12,2013-08-29 11:43:00,2013-08-29,2013-08-29,72.0,69.0,10.0,7.0,0,4.0,NaN,320.0,95113.0,2013-08-29,2013-08-29
